# HUVTSP - VANGUARD DEFENSE INTERNSHIP - WEEK 3

## Install Dependencies

In [ ]:
import kagglehub
import os
import shutil
!pip install split-folders -q
import splitfolders

In [ ]:
# Download latest version
path = kagglehub.dataset_download("a2015003713/militaryaircraftdetectiondataset")

print("Path to dataset files:", path)

## Filtering Selected Aircraft Types

In [ ]:
source_dir = f'{path}/crop/'
destination_dir = '/content/dataset/'
subfolders_to_copy = ['F15', 'F16', 'F18', 'F22', 'F35', 'B1', 'B2', 'C17', 'C130']

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Copy each specified subfolder
for folder in subfolders_to_copy:
    source_path = os.path.join(source_dir, folder)
    destination_path = os.path.join(destination_dir, folder)
    if os.path.exists(source_path):
        shutil.copytree(source_path, destination_path)
        print(f"Copied {folder} to {destination_dir}")
    else:
        print(f"Source folder {folder} not found in {source_dir}")
        raise FileNotFoundError

print("Copying complete.")

## Automaticall Labeling Aircrafts

In [ ]:
aircrafts = ["F15", "F16", "F18", "F22", "F35", "B1", "B2", "C17", "C130"]
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

for i, aircraft in enumerate(aircrafts):

  image_directory = f"/content/dataset/{aircraft}"
  label_directory = "/content/dataset/labels"
  CLASS_ID = i

  os.makedirs(label_directory, exist_ok=True)

  yolo_label_content = f"{CLASS_ID} 0.5 0.5 1.0 1.0"

  print(f"▶️ Reading images from: {image_directory}")
  print("-" * 30)

  if not os.path.isdir(image_directory):
      continue
  else:
      for filename in os.listdir(image_directory):
          if os.path.splitext(filename)[1].lower() in image_extensions:
              label_filename = os.path.splitext(filename)[0] + ".txt"
              label_filepath = os.path.join(label_directory, label_filename)
              try:
                  with open(label_filepath, 'w') as f:
                      f.write(yolo_label_content)
              except IOError as e:
                  print(f"❌ Error creating label for {filename}: {e}")

## Collect All Images in One Directory

In [ ]:
all_images_destination_dir = '/content/dataset/images/'
os.makedirs(all_images_destination_dir, exist_ok=True)


print("Starting image transfer to a single folder...")
for aircraft in aircrafts:
    source_image_directory = f"/content/dataset/{aircraft}"
    print(f"Processing images from: {source_image_directory}")
    if os.path.isdir(source_image_directory):
        for filename in os.listdir(source_image_directory):
            if os.path.splitext(filename)[1].lower() in image_extensions:
                source_filepath = os.path.join(source_image_directory, filename)
                destination_filepath = os.path.join(all_images_destination_dir, filename)
                try:
                    shutil.move(source_filepath, destination_filepath)
                except IOError as e:
                    print(f"❌ Error moving {filename}: {e}")
    else:
        print(f"Source directory not found: {source_image_directory}")

print("Image transfer complete.")

## Delete Old Folders

In [ ]:
for aircraft in aircrafts:
    folder_path = f"/content/dataset/{aircraft}"
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        shutil.rmtree(folder_path)
        print(f"Deleted folder: {folder_path}")
    else:
        print(f"Folder not found or not a directory: {folder_path}")

## Train - Validation Split

In [ ]:
input_folder = '/content/dataset/'

output_folder = '/content/split/'

splitfolders.ratio(input_folder,
                   output=output_folder,
                   seed=42,
                   ratio=(.8, .2),
                   group_prefix=None,
                   move=False) # Set to True to move files instead of copying

print("Dataset splitting complete!")

## dataset.yaml File

Create this dataset.yaml - content:

train: /content/split/train/images

val: /content/split/val/images

nc: 9

names: ["F15", "F16", "F18", "F22", "F35", "B1", "B2", "C17", "C130"]